In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
import pandas as pd
import numpy as np
import math
import os
import shutil

In [23]:
def createfolder(save_path , fp_name , DOI):
    r_path = os.path.join(save_path , fp_name)
    fp_path = os.path.join(r_path , 'FP')
    cited_path = os.path.join(r_path , 'cited')
    citing_path = os.path.join(r_path , 'citing')
    DOI_file = os.path.join(r_path , 'DOI.txt')
    os.mkdir(r_path)
    os.mkdir(fp_path)
    os.mkdir(cited_path)
    os.mkdir(citing_path)
        
    with open(DOI_file , mode='w') as f:
        f.write(DOI)
    return r_path , fp_path , cited_path , citing_path

In [24]:
def searchDOI(browser , DOI):
    #选择核心合集
    browser.find_element_by_xpath('//*[@id="snSelectDb"]/button/span[1]').click()
    time.sleep(0.5)
    browser.find_element_by_xpath('//*[@id="global-select"]/div[1]/div[2]/div[2]').click()
    time.sleep(0.5)
    browser.find_element_by_xpath('//*[@id="snSearchType"]/div[1]/app-search-row/div/div[1]/app-select-search-field/wos-select/button').click()
    #检索选择DOI
    time.sleep(0.5)
    browser.find_element_by_xpath('//*[@id="global-select"]/div[1]/input').send_keys('DOI')
    time.sleep(0.5)
    browser.find_element_by_xpath('//*[@id="global-select"]/div[1]/div[2]/div').click()
    #键入DOI
    time.sleep(0.5)
    browser.find_element_by_xpath('//*[@id="snSearchType"]/div[1]/app-search-row/div/div[2]/input').clear()
    browser.find_element_by_xpath('//*[@id="snSearchType"]/div[1]/app-search-row/div/div[2]/input').send_keys(DOI)
    time.sleep(0.5)
    browser.find_element_by_xpath('//*[@id="snSearchType"]/div[3]/button[2]/span[1]').click()
    time.sleep(2)

In [40]:
def export(browser , left , right):
    time.sleep(.5)
    #选择导出
    browser.find_element_by_xpath('//*[@id="snRecListTop"]/app-export-menu/div/button').click()
    time.sleep(.5)
    #选择纯文本
    browser.find_element_by_xpath('//*[@id="exportToFieldTaggedButton"]').click()
    time.sleep(1)
    #勾选radio框
    try:
        browser.find_element_by_xpath('//*[@id="radio3"]').send_keys(Keys.SPACE)
        time.sleep(.5)
        browser.find_element_by_xpath('//*[@id="radio3"]').click()
        time.sleep(.5)
    except:
        browser.find_element_by_xpath('//*[@id="radio3-input"]').send_keys(Keys.SPACE)
        time.sleep(.5)
        browser.find_element_by_xpath('//*[@id="radio3-input"]').click()
        time.sleep(.5)
    
    #输入条数（一次性导出多少个）
    try:
        browser.find_element_by_xpath('//input[@aria-label="Input starting record range"]').clear()
        browser.find_element_by_xpath('//input[@aria-label="Input starting record range"]').send_keys(str(left))
        time.sleep(.5)
        browser.find_element_by_xpath('//input[@aria-label="Input ending record range"]').clear()
        browser.find_element_by_xpath('//input[@aria-label="Input ending record range"]').send_keys(str(right))
        time.sleep(.5)
    except:
        browser.find_element_by_xpath('//input[@id="mat-input-0"]').clear()
        browser.find_element_by_xpath('//input[@id="mat-input-0"]').send_keys(str(left))
        time.sleep(.5)
        browser.find_element_by_xpath('//input[@id="mat-input-1"]').clear()
        browser.find_element_by_xpath('//input[@id="mat-input-1"]').send_keys(str(right))
        time.sleep(.5)
    #选择全记录与引文
    #browser.find_element_by_xpath('/html/body/app-wos/div/div/main/app-input-route/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/div/div[1]/wos-select/button').click()
    browser.find_element_by_xpath('//button[@aria-haspopup="listbox"]').click()
    time.sleep(.5)
    browser.find_elements_by_xpath('//div[@aria-label="Options dropdown"]/div')[-1].click()
    time.sleep(.5)
    #导出export
    browser.find_element_by_xpath('//button[@class="mat-focus-indicator cdx-but-md mat-stroked-button mat-button-base mat-primary"]').click()
    time.sleep(2 + (right - left)*0.01)

In [26]:
def ref_cunt(browser):
    num = browser.find_element_by_xpath('//span[@class="brand-blue"]').text
    if len(num) <= 3:
        return int(num)
    else:
        return int(num[len(num) - 5] + num[len(num) - 3:])

def export_data_mini(browser , cunt):
    num = ref_cunt(browser)
    itra = math.floor(num/500) + 1
    for itr in range(itra):
        if itr != (itra - 1):
            left = 500 * itr + 1
            right = 500 * (itr + 1)
        else:
            left = 500 * itr + 1
            right = num

        export(browser , left , right)
        for i in range(25):
            if len(os.listdir(download_path)) <= cunt:
                time.sleep(2)
                try:
                    browser.find_element_by_xpath('//button[@class="mat-focus-indicator cdx-but-md mat-stroked-button mat-button-base mat-primary"]').click()
                except:
                        pass
                
        if len(os.listdir(download_path)) > cunt:
            cunt += 1

In [27]:
def export_data(browser , error_cunt , error_lst , cunt , url):
    try:
        num = ref_cunt(browser)
        itra = math.ceil(num/500)
        for itr in range(itra):
            if itr != (itra - 1):
                left = 500 * itr + 1
                right = 500 * (itr + 1)
            else:
                left = 500 * itr + 1
                right = num
            export(browser , left , right)

            for i in range(25):
                if len(os.listdir(download_path)) <= cunt:
                    time.sleep(2)
                    try:
                        browser.find_element_by_xpath('//button[@class="mat-focus-indicator cdx-but-md mat-stroked-button mat-button-base mat-primary"]').click()
                    except:
                        pass

            if len(os.listdir(download_path)) > cunt:
                cunt += 1
        return error_cunt , error_lst , cunt
                
    except:
        error_cunt += 1
        error_lst.append(url)
        return error_cunt , error_lst , cunt

In [28]:
def page_scroll(browser):
    # 重复控制滚轮滑动，以获得全部URL数据
    for i in range(100):
        browser.execute_script("window.scrollBy(0,200)")
        time.sleep(0.1)

In [29]:
# 翻页
def page_turning(browser):
    browser.find_elements_by_xpath('//button[@data-ta="next-page-button"]')[1].click()
    time.sleep(0.5)

In [30]:
#移动文件，将下载路径的文件全部移动至保存路径
def movefile(srcfile , dstfile):
    file_lst = os.listdir(srcfile)
    for file in file_lst:
        srcpath = os.path.join(srcfile , file)
        shutil.move(srcpath , dstfile)         #移动文件
        
#movefile(download_path , cited_path)

# 分三部分爬取：

# 1 FP

In [31]:
def get_fp_data(browser , download_path , fp_path , DOI):
    searchDOI(browser , DOI)                #自行登录账号，选择WOS核心合集，并搜索指定论文FP
    time.sleep(5)
    export(browser , 1 , 1)
    
    for i in range(25):
        if len(os.listdir(download_path)) != 1:
            time.sleep(2)
            try:
                browser.find_element_by_xpath('//button[@class="mat-focus-indicator cdx-but-md mat-stroked-button mat-button-base mat-primary"]').click()
            except:
                pass
            
    movefile(download_path , fp_path)

# 2 CITING

In [32]:
def get_citing_data(browser , download_path , citing_path , DOI):
    try:
        time.sleep(1)
        browser.find_element_by_xpath('//a[@id="headerLogo"]').click()  #重新回到WOS主页面
    except:
        time.sleep(10)
        browser.find_element_by_xpath('//a[@id="headerLogo"]').click()  #重新回到WOS主页面
        
    time.sleep(3)
    searchDOI(browser , DOI)
    time.sleep(2)
    
    cunt = 0
    try:
        browser.find_elements_by_xpath('//div[@class="citations ng-star-inserted"]/a')[0].click()#进入被引文献界面
    except:
        time.sleep(10)
        browser.find_elements_by_xpath('//div[@class="citations ng-star-inserted"]/a')[0].click()#进入被引文献界面
        
    time.sleep(2)
    export_data_mini(browser , cunt)
    movefile(download_path , citing_path)

# 3 CITED

In [42]:
def get_cited_data(browser , download_path , cited_path , DOI):
    try:
        time.sleep(1)
        browser.find_element_by_xpath('//a[@id="headerLogo"]').click()  #重新回到WOS主页面
    except:
        time.sleep(10)
        browser.find_element_by_xpath('//a[@id="headerLogo"]').click()  #重新回到WOS主页面

    time.sleep(3)
    searchDOI(browser , DOI)
    time.sleep(2)
    try:
        browser.find_elements_by_xpath('//div[@class="link-container ref-count ng-star-inserted"]/a')[0].click()#进入参考文献界面
    except:
        time.sleep(10)
        browser.find_elements_by_xpath('//div[@class="link-container ref-count ng-star-inserted"]/a')[0].click()#进入参考文献界面
    time.sleep(2)

    #获取所有参考文献的被引页面URL
    total_lst = []
    page_num = int(browser.find_element_by_xpath('//span[@class="end-page ng-star-inserted"]').text)
    for page in range( page_num ):
        #获取参考文献的全部被引页面（仅针对存在被引的情况下）
        page_scroll(browser)
        lst = []
        for i in browser.find_elements_by_xpath('//div[@class="citations ng-star-inserted"]/a'):
            lst.append(i.get_attribute('href'))
        if page < page_num -1:
            page_turning(browser)
        total_lst.extend(lst)
    
    
    cunt = 0
    total_cunt = 0
    error_cunt = 0
    error_lst = []
    for url in total_lst:
        browser.get(url)
        time.sleep(3)
        error_cunt , error_lst , cunt = export_data(browser , error_cunt , error_lst , cunt , url)
        total_cunt += 1
    
    error_cunt2 = 0
    error_lst2 = []
    for url in error_lst:
        browser.get(url)
        time.sleep(5)
        error_cunt2 , error_lst2 , cunt = export_data(browser , error_cunt2 , error_lst2 , cunt , url)
        
    error_cunt3 = 0
    error_lst3 = []
    for url in error_lst2:
        browser.get(url)
        time.sleep(10)
        error_cunt3 , error_lst3 , cunt = export_data(browser , error_cunt3 , error_lst3 , cunt , url)
        
    movefile(download_path , cited_path)
    
    return total_cunt , error_cunt3 , error_lst3

In [34]:
def get_data(browser , save_path , download_path , DOI , fp_name):
    r_path , fp_path , cited_path , citing_path = createfolder(save_path , fp_name , DOI)
    get_fp_data(browser , download_path , fp_path , DOI)
    print('FP数据采集顺利完成')
    get_citing_data(browser , download_path , citing_path , DOI)
    print('citing数据采集顺利完成')
    total_cunt , error_cunt , error_lst = get_cited_data(browser , download_path , cited_path , DOI)
    print('参考文献总数量：', total_cunt , '  错误的参考文献数量：', error_cunt)
    print('错误参考文献链接：', error_lst)
    save_cited(total_cunt - error_cunt , save_path , fp_name)
    print('cited数据采集顺利完成')

In [35]:
def save_cited(cunt , save_path , fp_name):
    save_cited_path = os.path.join(save_path , fp_name , 'RCount.txt')
    with open(save_cited_path , 'w') as f:
        f.write(str(cunt))

In [36]:
def main():
    for i in range(len(DOI_lst)):
        get_data(browser , save_path , download_path , DOI_lst[i] , fp_name_lst[i])
        print('最终完成！', DOI_lst[i] , fp_name_lst[i])
        print('-'*20)
        try:
            time.sleep(1)
            browser.find_element_by_xpath('//a[@id="headerLogo"]').click()  #重新回到WOS主页面
        except:
            time.sleep(10)
            browser.find_element_by_xpath('//a[@id="headerLogo"]').click()  #重新回到WOS主页面

# 建立保存文件的文件夹

In [16]:
# 确定文件下载和保存的地址,一定要提前清空下载地址的文件,一定要在同一个磁盘驱动器
download_path = r'C:\Users\Star\downloads'
save_path = r'C:\Users\Star\Music\SCIENTOMETIRCS'

In [17]:
with open(r'C:\Users\Star\Desktop\SCIENTOMETRICS100.txt', 'r', encoding='utf-8') as f:
    data = (f.readlines()[2:])
    
title_lst = []
doi_lst = []
paper_lst = []

b0 = 0
b1 = 0

for d in data:
    if d[:2] == 'TI':
        b0 = 1
        title_lst.append(d[3:-1])
    if d[:2] == 'DI':
        b1 = 1
        doi_lst.append(d[3:-1])
    if d == '\n':
        if b0 == 0 and b1 == 1:
            doi_lst.pop()
        elif b0 == 1 and b1 == 0:
            title_lst.pop()
        else:
            pass
        b0 = 0
        b1 = 0

# 启用chrome-browser，进入无图片模式

In [18]:
chrome_opt = webdriver.ChromeOptions()
prefs = {'profile.managed_default_content_settings.images': 2}
chrome_opt.add_experimental_option('prefs',prefs)
browser = webdriver.Chrome(chrome_options=chrome_opt)

C:\Users\Star\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


# 进入搜索页面

In [19]:
url = "https://www.webofscience.com/wos/alldb/basic-search"
browser.get(url)
wait = WebDriverWait(browser, 5)

自行登录账号，并且调整为英文界面

In [43]:
fp_name_lst = [str(i) for i in range(1,204)][1:]
DOI_lst = doi_lst[1:]

In [45]:
if __name__ == '__main__':
    main()

FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 13   错误的参考文献数量： 1
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:000270672300008?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.1007/s11192-006-0144-7 2
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 26   错误的参考文献数量： 0
错误参考文献链接： []
cited数据采集顺利完成
最终完成！ 10.1007/s11192-015-1765-5 3
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 20   错误的参考文献数量： 1
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1977CS22100002?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.1007/BF02019280 4
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 59   错误的参考文献数量： 1
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:000250317700079?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.1007/s11192-011-0494-7 5
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 39   错误的参考文献数量： 0
错误参考文献链接： []
cited数据采集顺利完成
最终完

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="snRecListTop"]/app-export-menu/div/button"}
  (Session info: chrome=96.0.4664.45)


In [46]:
fp_name_lst = [str(i) for i in range(1,204)][21:]
DOI_lst = doi_lst[21:]
if __name__ == '__main__':
    main()

FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 102   错误的参考文献数量： 7
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1981NG70300002?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1980KC11100001?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1979HE55200002?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1978FW13900005?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1978GF69700006?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1969F015300007?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1969F015300009?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.10

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="snRecListTop"]/app-export-menu/div/button"}
  (Session info: chrome=96.0.4664.45)


In [47]:
fp_name_lst = [str(i) for i in range(1,204)][29:]
DOI_lst = doi_lst[29:]
if __name__ == '__main__':
    main()

FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 4   错误的参考文献数量： 1
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1980JJ16200003?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.1007/BF02093973 30
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 30   错误的参考文献数量： 2
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1979HY50300004?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1977DH67000001?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.1007/BF02018100 31
--------------------


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@id="headerLogo"]"}
  (Session info: chrome=96.0.4664.45)


In [48]:
fp_name_lst = [str(i) for i in range(1,204)][31:]
DOI_lst = doi_lst[31:]
if __name__ == '__main__':
    main()

FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 24   错误的参考文献数量： 0
错误参考文献链接： []
cited数据采集顺利完成
最终完成！ 10.1007/s11192-017-2300-7 32
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 43   错误的参考文献数量： 1
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:000443601300013?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.1007/s11192-015-1645-z 33
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 29   错误的参考文献数量： 0
错误参考文献链接： []
cited数据采集顺利完成
最终完成！ 10.1007/s11192-005-0007-7 34
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 34   错误的参考文献数量： 2
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1995QR06900021?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1963CHH9100010?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.1007/s11192-006-0176-z 35
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 5   错误的参考文献数量： 0
错误参考文献链接： []
cited

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="snRecListTop"]/app-export-menu/div/button"}
  (Session info: chrome=96.0.4664.45)


In [ ]:
#29 , 73的DOI不对

In [53]:
fp_name_lst = [str(i) for i in range(1,204)][53:]
DOI_lst = doi_lst[51:]
if __name__ == '__main__':
    main()

FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 11   错误的参考文献数量： 1
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:000270672300008?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.1007/s11192-006-0143-8 54
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 9   错误的参考文献数量： 0
错误参考文献链接： []
cited数据采集顺利完成
最终完成！ 10.1007/BF02459299 55
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 25   错误的参考文献数量： 2
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1981QU48000002?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1980JY31100002?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.1007/BF02129599 56
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 20   错误的参考文献数量： 1
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1968C591600001?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.1007

ElementClickInterceptedException: Message: element click intercepted: Element <a id="headerLogo" cdxanalyticsevent="" cdxanalyticsaction="Click" cdxanalyticscategory="wos-header-home" class="logo font-size-24 ng-star-inserted" aria-label="Web of Science header logo. Click to return to home page" href="/wos/">...</a> is not clickable at point (386, 72). Other element would receive the click: <div class="dimmed-background"></div>
  (Session info: chrome=96.0.4664.45)


In [54]:
fp_name_lst = [str(i) for i in range(1,204)][63:]
DOI_lst = doi_lst[63:]
if __name__ == '__main__':
    main()

FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 54   错误的参考文献数量： 5
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:000071644000004?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/BCI:BCI199293117864?from=bci&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1989R868800005?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1969D381500070?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1966ZE37100005?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.1007/s11192-008-1939-5 64
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 234   错误的参考文献数量： 33
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1989R834000004?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscie

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="snRecListTop"]/app-export-menu/div/button"}
  (Session info: chrome=96.0.4664.45)


In [56]:
fp_name_lst = [str(i) for i in range(1,204)][73:]
DOI_lst = doi_lst[73:]
if __name__ == '__main__':
    main()

FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 47   错误的参考文献数量： 0
错误参考文献链接： []
cited数据采集顺利完成
最终完成！ 10.1007/s11192-007-1971-x 74
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 39   错误的参考文献数量： 4
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:000305337100003?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:000305337100083?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:000305337100047?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:000443601300013?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.1007/s11192-011-0580-x 75
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 55   错误的参考文献数量： 3
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:000235837800016?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.web

ElementClickInterceptedException: Message: element click intercepted: Element <a id="headerLogo" cdxanalyticsevent="" cdxanalyticsaction="Click" cdxanalyticscategory="wos-header-home" class="logo font-size-24 ng-star-inserted" aria-label="Web of Science header logo. Click to return to home page" href="/wos/">...</a> is not clickable at point (386, 72). Other element would receive the click: <div class="dimmed-background"></div>
  (Session info: chrome=96.0.4664.45)


In [57]:
fp_name_lst = [str(i) for i in range(1,204)][91:]
DOI_lst = doi_lst[91:]
if __name__ == '__main__':
    main()

FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 14   错误的参考文献数量： 0
错误参考文献链接： []
cited数据采集顺利完成
最终完成！ 10.1007/s11192-010-0306-5 92
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 17   错误的参考文献数量： 3
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1978GA02500008?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1977ET13500001?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1968A419500007?from=woscc&type=colluid&eventMode=timeCitedOnSummary']
cited数据采集顺利完成
最终完成！ 10.1007/s11192-007-1777-2 93
--------------------
FP数据采集顺利完成
citing数据采集顺利完成
参考文献总数量： 19   错误的参考文献数量： 6
错误参考文献链接： ['https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1979GL58000001?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.webofscience.com/wos/woscc/citing-summary/WOS:A1978FQ22300142?from=woscc&type=colluid&eventMode=timeCitedOnSummary', 'https://www.web

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="snRecListTop"]/app-export-menu/div/button"}
  (Session info: chrome=96.0.4664.45)


In [62]:
fp_name_lst = [str(i) for i in range(1,204)][106:]
DOI_lst = doi_lst[106:]
if __name__ == '__main__':
    main()

ElementClickInterceptedException: Message: element click intercepted: Element <div tabindex="-1" class="wrap-mode ng-star-inserted" role="option" aria-selected="false" aria-label="Web of Science Core Collection">...</div> is not clickable at point (761, 368). Other element would receive the click: <p id="pendo-text-577a02d1" class="bb-text _pendo-text-plain" data-_pendo-text-plain-3="" style="display: block; color: inherit; font-size: inherit; letter-spacing: inherit; line-height: inherit; overflow-wrap: break-word; padding: 0px; position: relative; text-transform: inherit; width: auto; font-style: inherit; text-align: inherit; margin: 0px; float: none; vertical-align: baseline; white-space: pre-wrap;">...</p>
  (Session info: chrome=96.0.4664.45)
